In [ ]:
import os
import cv2
import numpy as np
from detectron2.utils.visualizer import Visualizer, ColorMode
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data.datasets import register_coco_instances
from detectron2.data import build_detection_test_loader
from detectron2.structures import Boxes, pairwise_iou


# Step 1: Register your test dataset
camNo = "Cam604"
data_path = r"C:\Users\choni\Pictures\Merged image"
print(data_path)
annotation_file = f"{data_path}\\{camNo}_dec_16_coco.json" # COCO-format JSON file
image_dir = f"{data_path}\\images\\"  # Directory containing test images
print(annotation_file)
register_coco_instances("test_dataset_name", {}, annotation_file, image_dir)


# Step 2: Load configuration and model

cfg = get_cfg()
model_path = "models\\March 2025\\Feb_March_Day_Night_15000_iters_v2" 
                                                                      
cfg.merge_from_file(f"{model_path}/config.yml")  # Path to config file
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 #0.6
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.4 #0.3
cfg.MODEL.WEIGHTS = f"{model_path}/model_best.pth"  # Path to trained model weights
predictor = DefaultPredictor(cfg)

# Step 3: Load the dataset and metadata
dataset_dicts = DatasetCatalog.get("test_dataset_name")
metadata = MetadataCatalog.get("test_dataset_name")

# Step 4: Define IoU threshold for matching predictions with ground truth
iou_threshold = 0.3

output_dir = f"{data_path}/detections_removed_mask"
os.makedirs(output_dir, exist_ok=True)

#E:\Nyi Zaw Aung\Sumiyoshi_annotation\Testing Annotation\16.12.2024\Resting Area\Cam602\images
#E:\Nyi Zaw Aung\Sumiyoshi_annotation\Testing Annotation\16.12.2024\Resting Area\Cam602\images

In [ ]:

def save_precision_recall_accuracy_in_csv(precision, recall, accuracy, output_dir):
    with open(f"{output_dir}/{camNo}.csv", "w") as f:
        f.write("Precision, Recall, Accuracy\n")
        f.write(f"{precision}, {recall}, {accuracy}\n")
        
        
def save_precision_recall_accuracy_APs_in_csv(precision, recall, accuracy, results,total_tp,total_fp,total_fn,total_gt,total_pred, output_dir):
    segms = results.get("segm",{})
    with open(f"{output_dir}/{camNo}.csv", "w") as f:
        f.write("Precision, Recall, Accuracy, TP,FP,FN,GT_total,Pred_total,AP, AP50, AP75\n")
        f.write(f"{precision}, {recall}, {accuracy},{total_tp},{total_fp},{total_fn},{total_gt},{total_pred}, {segms.get("AP",None)} , {segms.get("AP50",None)}, {segms.get("AP75",None)}\n")

In [ ]:

# Initialize counters for TP, FP, FN
total_tp = 0
total_fp = 0
total_fn = 0
total_gt = 0
total_pred = 0
for d in dataset_dicts:
    img_path = d["file_name"]
    img = cv2.imread(img_path)
    #print(img_path)
    # Skip invalid or non-JPG files
    if img is None or ".jpg" not in d["file_name"]:
        continue
    
    # Get ground truth annotations (bounding boxes)
    gt_boxes = [ann["bbox"] for ann in d["annotations"] if "bbox" in ann]
    gt_boxes = Boxes(torch.tensor(gt_boxes)) if len(gt_boxes) > 0 else Boxes(torch.empty(0, 4))  # Handle empty cases
    # Get predictions
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    
    pred_boxes = instances.pred_boxes if instances.has("pred_boxes") else None
    
    # Count detections and ground truth boxes
    num_predictions = len(pred_boxes) if pred_boxes is not None else 0
    num_ground_truths = len(gt_boxes)
    total_gt += num_ground_truths
    total_pred += num_predictions
    # Compare predictions with ground truth
    bad_flag = False  # Flag to mark if the image is "bad"
    has_iou = True
    if pred_boxes is not None:
        iou_matrix = pairwise_iou(pred_boxes, gt_boxes) if num_ground_truths > 0 else torch.zeros((num_predictions, 0))
        
        # Match predictions to ground truth using IoU threshold (e.g., 0.5)
        iou_threshold = 0.5
        matched_gt_indices = set()
        #print(img_path)
        
        if iou_matrix.numel() == 0:
            has_iou = False
        else:
            for pred_idx in range(iou_matrix.shape[0]):
                max_iou_idx = iou_matrix[pred_idx].argmax()
                max_iou = iou_matrix[pred_idx][max_iou_idx] if num_ground_truths > 0 else 0
                
                if max_iou >= iou_threshold:
                    matched_gt_indices.add(max_iou_idx)
            
            # Check for missing or extra detections
            unmatched_gt_count = num_ground_truths - len(matched_gt_indices)  # Ground truths not matched by any prediction
            unmatched_pred_count = num_predictions - len(matched_gt_indices)  # Predictions not matched to any ground truth
            
            if unmatched_gt_count > 0 or unmatched_pred_count > 0:
                bad_flag = True
    
    bad_flag = num_ground_truths != num_predictions
    if num_predictions > num_ground_truths :
        total_fp += num_predictions - num_ground_truths
        total_tp += num_ground_truths
    elif num_ground_truths > num_predictions:
        total_fn += num_ground_truths - num_predictions
        total_tp += num_predictions
    else:
        total_tp += num_predictions
        
    
    #elif num_ground_truths > 0:
    #    bad_flag = True  # If there are no predictions but ground truths exist
    
    #elif num_predictions > 0:
    #    bad_flag = True  # If there are predictions but no ground truths
    
    # Visualize predictions (bounding boxes and masks)
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1)  # Use grayscale for better visibility
    out = v.draw_instance_predictions(instances)
    
    # Save the image with predictions and add "bad" to the file name if applicable
    base_filename = os.path.splitext(os.path.basename(img_path))[0]
    suffix = f"_det{num_predictions}_gt{num_ground_truths}"
    
    if bad_flag:
        suffix += "_bad"
    
    output_path = os.path.join(output_dir, f"{base_filename}{suffix}.jpg")
    #if num_ground_truths > 0 or num_predictions > 0:
    cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

print("Processing complete! All images have been saved.")

In [ ]:
precision = total_tp / (total_tp + total_fp) 
recall = total_tp / (total_tp + total_fn)
accuracy = total_tp / (total_tp + total_fp + total_fn)
#total_tp = 0
#total_fp = 0
#total_fn = 0
#total_gt = 0
#total_pred = 0
print(f"Precision, {precision}, accuracy {recall}, accuracy {accuracy}")
    

In [ ]:
# Create evaluator and test loader
evaluator = COCOEvaluator("test_dataset_name", cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "test_dataset_name")

# Perform evaluation
results = inference_on_dataset(predictor.model, val_loader, evaluator)
#print(results)  # This will include precision and recall metrics

save_precision_recall_accuracy_APs_in_csv(precision, recall, accuracy,results, total_tp,total_fp,total_fn,total_gt,total_pred,output_dir)

if False:
    for d in dataset_dicts:
        img_path = d["file_name"]
        img = cv2.imread(img_path)
        
        # Skip invalid or non-JPG files
        if img is None or ".jpg" not in d["file_name"]:
            continue
        
        # Get predictions
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Visualize predictions (including masks)
        v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE_BW)  # Use grayscale for better visibility
        out = v.draw_instance_predictions(instances)
        
        # Draw yellow masks manually
        if instances.has("pred_masks"):
            pred_masks = instances.pred_masks.numpy()  # Shape: (N, H, W), where N is the number of detected objects
            
            for mask in pred_masks:
                # Create a yellow overlay for the mask
                yellow_mask = np.zeros_like(img, dtype=np.uint8)
                yellow_mask[:, :, 1] = 255  # Set green channel to max (R=0, G=255, B=0 for yellow)
                yellow_mask[:, :, 2] = 255  # Set red channel to max (R=255, G=255, B=0 for yellow)

                # Blend the mask with the original image
                mask_indices = mask.astype(bool)  # Convert binary mask to boolean for indexing
                img[mask_indices] = cv2.addWeighted(img, 0.5, yellow_mask, 0.5, 0)[mask_indices]
        
        # Save the image with predictions and yellow masks
        output_path = os.path.join(output_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, img)

In [ ]:


# Replace with your dataset paths


